In [1]:
import pandas as pd
import numpy as np
import sys
import os

In [2]:
""" Load the annotation file """

fpath = "/nfs/turbo/umms-indikar/shared/projects/DGC/data/scenic/motif2tf/motifs-v10nr_clust-nr.hgnc-m0.001-o0.0.tbl"

ann = pd.read_table(fpath, low_memory=False)
print(f"{ann.shape=}")
ann['motifs'] = ann['source_name'] + "__" + ann['motif_name']

print(f"{ann['motifs'].nunique()=}")
print()
print(f"{ann['gene_name'].nunique()=}")

# prepare the data for merging 
ann = ann[['#motif_id', 'motif_name', 'motif_description', 'source_name', 'gene_name', 'motifs']]
ann = ann.drop_duplicates() # this is just a check, doesn't do anything

ann.head()

ann.shape=(253096, 13)
ann['motifs'].nunique()=34584

ann['gene_name'].nunique()=1605


,#motif_id,motif_name,motif_description,source_name,gene_name,motifs
0,metacluster_196.3,EcR_usp,EcR/usp,bergman,HNF4A,bergman__EcR_usp
1,metacluster_196.3,EcR_usp,EcR/usp,bergman,HNF4G,bergman__EcR_usp
2,metacluster_196.3,EcR_usp,EcR/usp,bergman,NR1D1,bergman__EcR_usp
3,metacluster_196.3,EcR_usp,EcR/usp,bergman,NR1D2,bergman__EcR_usp
4,metacluster_196.3,EcR_usp,EcR/usp,bergman,NR1H2,bergman__EcR_usp


In [3]:
""" Load the score file """

# fpath = "/nfs/turbo/umms-indikar/shared/projects/DGC/data/scenic/motif_scores/hg38_10kbp_up_10kbp_down_full_tx_v10_clust.genes_vs_motifs.scores.feather"
fpath = "/nfs/turbo/umms-indikar/shared/projects/DGC/data/scenic/motif_scores/hg38_500bp_up_100bp_down_full_tx_v10_clust.genes_vs_motifs.scores.feather"

df = pd.read_feather(fpath)
print(f"{df.shape=}")
genes = df.columns.to_list()[:-1]

df.head()

df.shape=(5876, 27016)


,A1BG,A1BG-AS1,A1CF,A2M,A2M-AS1,A2ML1,A2MP1,A3GALT2,A4GALT,A4GNT,...,ZWINT,ZXDA,ZXDB,ZXDC,ZYG11A,ZYG11B,ZYX,ZZEF1,ZZZ3,motifs
0,0.0849,0.0,0.439,3.250,0.113,4.630,0.0161,0.0823,6.020,2.75000,...,3.450,4.320,4.3300,0.592,0.193,0.0399,0.336,0.0354,0.507,bergman__Su_H_
1,4.4000,0.0,10.100,3.400,0.000,2.090,4.6200,0.0293,8.610,0.00793,...,0.670,1.850,0.0000,0.803,2.540,0.3520,0.621,0.0000,0.363,bergman__croc
2,0.0000,0.0,0.347,0.257,0.000,3.800,0.1270,3.9700,0.248,0.20400,...,0.116,0.083,0.0821,0.245,0.160,0.0000,1.020,0.1140,3.990,bergman__pho
3,1.3300,0.0,3.520,0.242,0.000,0.824,8.8100,0.0000,1.610,0.09030,...,0.695,0.205,0.0883,4.900,0.443,0.0000,0.000,1.9700,0.704,bergman__tll
4,4.8700,0.0,1.660,3.010,0.392,2.920,1.2100,3.9700,4.780,2.28000,...,8.830,1.910,1.5500,8.370,8.400,1.7500,2.950,1.3600,7.030,c2h2_zfs__M0369


In [4]:
""" Create the merged object """

gf = pd.merge(ann[['gene_name', 'motifs']], df, 
              how='left',
              left_on='motifs',
              right_on='motifs')

print(f"{gf.shape=}")

gf.shape=(253096, 27017)


In [5]:
gx = gf.groupby('gene_name')[genes].mean()
print(f"{gx.shape=}")
gx = gx.fillna(0.00)
gx = gx.T # for indika!

gx.head()

gx.shape=(1605, 27015)


gene_name,ABL1,ACAA1,ADNP,ADNP2,AEBP2,AFF4,AHCTF1,AHDC1,AHR,AHRR,...,ZSCAN32,ZSCAN4,ZSCAN5A,ZSCAN5B,ZSCAN5C,ZSCAN9,ZXDA,ZXDB,ZXDC,ZZZ3
A1BG,1.65,0.0,5.060,0.0,0.0,8.470,6.400,2.290,0.83060,0.0,...,0.0,0.0,0.0,0.0,1.32,1.448,1.790,1.484333,0.0,1.37
A1BG-AS1,0.00,0.0,0.545,0.0,0.0,0.379,0.000,0.000,0.14640,0.0,...,0.0,0.0,0.0,0.0,0.00,0.770,1.465,1.032333,0.0,0.00
A1CF,1.55,0.0,0.000,0.0,0.0,3.780,4.880,1.092,0.31400,0.0,...,0.0,0.0,0.0,0.0,0.00,0.920,2.865,1.910000,0.0,2.67
A2M,1.39,0.0,0.827,0.0,0.0,4.700,0.810,1.055,0.27314,0.0,...,0.0,0.0,0.0,0.0,0.00,0.535,0.885,0.698000,0.0,3.45
A2M-AS1,0.00,0.0,0.000,0.0,0.0,1.450,0.111,0.000,0.00000,0.0,...,0.0,0.0,0.0,0.0,0.00,0.800,0.000,0.059333,0.0,0.00


In [6]:
outpath = "/nfs/turbo/umms-indikar/shared/projects/DGC/data/processed_data/"
outname = f"{outpath}500bp_up_100bp_down_B.csv"
gx.to_csv(outname, index=True)
print('done')

done
